In [ ]:
#@title 0. Install mr0
!wget https://github.com/MRsources/MRzero-Core/raw/main/documentation/playground_mr0/ptx_phantom.p &> /dev/null
!pip install git+https://gitlab.cs.fau.de/mrzero/pypulseq_rfshim.git@PyPulseq_rfshim_145_MR0 &> /dev/null
!pip install MRzeroCore &> /dev/null

(pulseq_ptx)=
# Pulseq with RF Shimming

In [ ]:
import MRzeroCore as mr0
import pypulseq as pp
import matplotlib.pyplot as plt
import numpy as np
import torch
import pickle

Load Phantom extendet with B1 Maps for pTx

In [ ]:
phantom = pickle.load(open("ptx_phantom.p", "rb"))

plt.figure(figsize=(8, 2), dpi=300)
for i in range(8):
    plt.subplot(2, 8, i+1)
    plt.imshow(phantom.B1[i,:,:,0].abs().T, vmin=0, origin="lower")
    plt.axis("off")
    plt.subplot(2, 8, i+9)
    plt.imshow(phantom.B1[i,:,:,0].angle().T, cmap="twilight", vmin=-np.pi, vmax=np.pi, origin="lower")
    plt.axis("off")
plt.show()

Sequence definition fixed part


In [ ]:
# %% S1. SETUP sys

## choose the scanner limits
system = pp.Opts(max_grad=28, grad_unit='mT/m', max_slew=150, slew_unit='T/m/s', rf_ringdown_time=20e-6,
              rf_dead_time=100e-6, adc_dead_time=20e-6, grad_raster_time=50 * 10e-6)

# %% S2. DEFINE the sequence
seq = pp.Sequence()

# Define FOV and resolution
# Values form MiniFLASH
fov = 192e-3
Nread = 32
Nphase = 32
slice_thickness = 5e-3  # slice propably in mm


# Define other gradients and ADC events
gx = pp.make_trapezoid(channel='x', flat_area=Nread/fov, flat_time=2e-3, system=system)
adc = pp.make_adc(num_samples=Nread, duration=gx.flat_time, delay=gx.rise_time, system=system)
gx_pre = pp.make_trapezoid(channel='x', area=-gx.area / 2, duration=5e-3, system=system)
gx_spoil = pp.make_trapezoid(channel='x', area=1.5*gx.area, duration=5e-3, system=system)

# ======
# CONSTRUCT SEQUENCE
# ======

experiment_id = 'FLASH_pTx'
rf_phase = 0
rf_inc = 0
rf_spoiling_inc=117

mode = 'cp' # @param ["cp", "ep"]
if mode == 'cp':
     shim_array = np.array([[0.35, 0    * np.pi / 180],
                   [0.35, -45  * np.pi / 180],
                   [0.35, -90  * np.pi / 180],
                   [0.35, -135 * np.pi / 180],
                   [0.35, -180 * np.pi / 180],
                   [0.35, 135  * np.pi / 180],
                   [0.35, 90   * np.pi / 180],
                   [0.35, 45   * np.pi / 180]], dtype=float)
else:
    shim_array = np.array([[0.35, 0    * np.pi / 180],
                   [0.35, -90  * np.pi / 180],
                   [0.35, 180  * np.pi / 180],
                   [0.35, 90   * np.pi / 180],
                   [0.35, 0    * np.pi / 180],
                   [0.35, -90  * np.pi / 180],
                   [0.35, -180 * np.pi / 180],
                   [0.35, 90   * np.pi / 180]], dtype=float)

rf, gz, gzr = pp.make_sinc_pulse(flip_angle=10 * np.pi / 180,
                                  system=system,
                                  duration=1e-3,
                                  slice_thickness=5e-3,
                                  apodization=0.5,
                                  time_bw_product=4,
                                  shim_array=shim_array,
                                  return_gz=True)



for ii in range(-Nphase // 2, Nphase // 2):

    rf.phase_offset = rf_phase / 180 * np.pi  # set current rf phase
    adc.phase_offset = rf_phase / 180 * np.pi  # follow with ADC

    rf_inc   = divmod(rf_inc + rf_spoiling_inc, 360.0)[1]  # increase increment
    rf_phase = divmod(rf_phase + rf_inc, 360.0)[1]  # increment additional phase

    #CP Line
    seq.add_block(rf, gz)
    seq.add_block(gzr)
    gy_pre = pp.make_trapezoid(channel='y', area=ii/fov, duration=pp.calc_duration(gx_pre), system=system)
    seq.add_block(gx_pre, gy_pre)
    seq.add_block(adc, gx)
    gy_spoil = pp.make_trapezoid(channel='y', area=-ii/fov, duration=pp.calc_duration(gx_pre), system=system)
    seq.add_block(gx_spoil, gy_spoil)
    if ii < Nphase - 1:
        seq.add_block(pp.make_delay(0.001))

# %% S3. CHECK, PLOT and WRITE the sequence  as .seq
ok, error_report = seq.check_timing()  # Check whether the timing of the sequence is correct
if ok:
    print('Timing check passed successfully')
else:
    print('Timing check failed. Error listing follows:')
    [print(e) for e in error_report]


# Prepare the sequence output for the scanner
seq.set_definition('FOV', [fov, fov, slice_thickness])
seq.set_definition('Name', experiment_id)
seq.write('external.seq')


In [ ]:
#seq_file = "flash pTx EP.seq" #@param ["flash pTx EP.seq", "flash pTx CP.seq", "flash pTx QM.seq"] {allow-input: true}

seq_file = 'external.seq'

data = phantom.build()

seq = mr0.Sequence.from_seq_file(seq_file)
seq.plot_kspace_trajectory()

# Simulate the sequence
graph = mr0.compute_graph(seq, data, 200, 1e-3)
signal = mr0.execute_graph(graph, seq, data)
reco = mr0.reco_adjoint(signal, seq.get_kspace(), resolution=(64, 64, 1), FOV=(1, 1, 1))

plt.figure(figsize=(7, 3), dpi=200)
plt.subplot(121)
plt.title("Magnitude")
plt.imshow(reco[:, :, 0].T.abs(), origin="lower", vmin=0, cmap="gray")
plt.colorbar()
plt.subplot(122)
plt.title("Phase")
plt.imshow(reco[:, :, 0].T.angle(), origin="lower", vmin=-np.pi, vmax=np.pi, cmap="twilight")
plt.colorbar()
plt.show()

all in one

In [ ]:
# %% S1. SETUP sys

## choose the scanner limits
system = pp.Opts(max_grad=28, grad_unit='mT/m', max_slew=150, slew_unit='T/m/s', rf_ringdown_time=20e-6,
              rf_dead_time=100e-6, adc_dead_time=20e-6, grad_raster_time=50 * 10e-6)

# %% S2. DEFINE the sequence
seq = pp.Sequence()

# Define FOV and resolution
# Values form MiniFLASH
fov = 200e-3
Nread = 32
Nphase = 32
slice_thickness = 5e-3  # slice propably in mm


# Define other gradients and ADC events
gx = pp.make_trapezoid(channel='x', flat_area=Nread/fov, flat_time=2e-3, system=system)
adc = pp.make_adc(num_samples=Nread, duration=gx.flat_time, delay=gx.rise_time, system=system)
gx_pre = pp.make_trapezoid(channel='x', area=-gx.area / 2, duration=5e-3, system=system)
gx_spoil = pp.make_trapezoid(channel='x', area=1.5*gx.area, duration=5e-3, system=system)

# ======
# CONSTRUCT SEQUENCE
# ======

experiment_id = 'FLASH_pTx'
rf_phase = 0
rf_inc = 0
rf_spoiling_inc=117

ch0_mag  = 0.11 # @param {type:"slider", min:0.05, max:1, step:0.01}
ch1_mag  = 0.1 # @param {type:"slider", min:0.05, max:1, step:0.01}
ch2_mag  = 0.32 # @param {type:"slider", min:0.05, max:1, step:0.01}
ch3_mag  = 0.31 # @param {type:"slider", min:0.05, max:1, step:0.01}
ch4_mag  = 0.11 # @param {type:"slider", min:0.05, max:1, step:0.01}
ch5_mag  = 0.12 # @param {type:"slider", min:0.05, max:1, step:0.01}
ch6_mag  = 0.36 # @param {type:"slider", min:0.05, max:1, step:0.01}
ch7_mag  = 0.11 # @param {type:"slider", min:0.05, max:1, step:0.01}

ch0_ph   = -2    # @param {type:"slider", min:-360, max:360, step:1}
ch1_ph   = 3  # @param {type:"slider", min:-360, max:360, step:1}
ch2_ph   = 3  # @param {type:"slider", min:-360, max:360, step:1}
ch3_ph   = 0 # @param {type:"slider", min:-360, max:360, step:1}
ch4_ph   = 3 # @param {type:"slider", min:-360, max:360, step:1}
ch5_ph   = 7  # @param {type:"slider", min:-360, max:360, step:1}
ch6_ph   = 8   # @param {type:"slider", min:-360, max:360, step:1}
ch7_ph   = 10   # @param {type:"slider", min:-360, max:360, step:1}


shim_array = np.array([[ch0_mag, ch0_ph * np.pi / 180],
                       [ch1_mag, ch1_ph * np.pi / 180],
                       [ch2_mag, ch2_ph * np.pi / 180],
                       [ch3_mag, ch3_ph * np.pi / 180],
                       [ch4_mag, ch4_ph * np.pi / 180],
                       [ch5_mag, ch5_ph * np.pi / 180],
                       [ch6_mag, ch6_ph * np.pi / 180],
                       [ch7_mag, ch7_ph * np.pi / 180]], dtype=float)

rf, gz, gzr = pp.make_sinc_pulse(flip_angle=10 * np.pi / 180,
                                  system=system,
                                  duration=1e-3,
                                  slice_thickness=5e-3,
                                  apodization=0.5,
                                  time_bw_product=4,
                                  shim_array=shim_array,
                                  return_gz=True)



for ii in range(-Nphase // 2, Nphase // 2):

    rf.phase_offset = rf_phase / 180 * np.pi  # set current rf phase
    adc.phase_offset = rf_phase / 180 * np.pi  # follow with ADC

    rf_inc   = divmod(rf_inc + rf_spoiling_inc, 360.0)[1]  # increase increment
    rf_phase = divmod(rf_phase + rf_inc, 360.0)[1]  # increment additional pahse

    #CP Line
    seq.add_block(rf, gz)
    seq.add_block(gzr)
    gy_pre = pp.make_trapezoid(channel='y', area=ii/fov, duration=pp.calc_duration(gx_pre), system=system)
    seq.add_block(gx_pre, gy_pre)
    seq.add_block(adc, gx)
    gy_spoil = pp.make_trapezoid(channel='y', area=-ii/fov, duration=pp.calc_duration(gx_pre), system=system)
    seq.add_block(gx_spoil, gy_spoil)
    if ii < Nphase - 1:
        seq.add_block(pp.make_delay(0.001))

# %% S3. CHECK, PLOT and WRITE the sequence  as .seq
ok, error_report = seq.check_timing()  # Check whether the timing of the sequence is correct
if ok:
    print('Timing check passed successfully')
else:
    print('Timing check failed. Error listing follows:')
    [print(e) for e in error_report]


# Prepare the sequence output for the scanner
seq.set_definition('FOV', [fov, fov, slice_thickness])
seq.set_definition('Name', experiment_id)
seq.write('external.seq')

seq_file = 'external.seq'

data = phantom.build()

seq = mr0.Sequence.from_seq_file(seq_file)
#seq.plot_kspace_trajectory()

# Create a non-pTx version of the sequence
prepass_seq = seq.clone()
for rep in prepass_seq:
    rep.pulse.angle = rep.pulse.angle.mean() / np.sqrt(1 / rep.pulse.angle.numel())
    rep.pulse.phase = rep.pulse.phase.mean()

# Simulate the sequence
graph = mr0.compute_graph(prepass_seq, data, 200, 1e-3)
signal = mr0.execute_graph(graph, seq, data)
reco = mr0.reco_adjoint(signal, seq.get_kspace(), resolution=(64, 64, 1), FOV=(1, 1, 1))

plt.figure(figsize=(7, 3), dpi=200)
plt.subplot(121)
plt.title("Magnitude")
plt.imshow(reco[:, :, 0].T.abs(), origin="lower", vmin=0, cmap="gray")
plt.colorbar()
plt.subplot(122)
plt.title("Phase")
plt.imshow(reco[:, :, 0].T.angle(), origin="lower", vmin=-np.pi, vmax=np.pi, cmap="twilight")
plt.colorbar()
plt.show()


In [ ]:
B1 = torch.zeros((64, 64), dtype=torch.cfloat)
B1 += shim_array[0, 0] * np.exp(1j * shim_array[0, 1]) * phantom.B1[0, :, :, 0]
B1 += shim_array[1, 0] * np.exp(1j * shim_array[1, 1]) * phantom.B1[1, :, :, 0]
B1 += shim_array[2, 0] * np.exp(1j * shim_array[2, 1]) * phantom.B1[2, :, :, 0]
B1 += shim_array[3, 0] * np.exp(1j * shim_array[3, 1]) * phantom.B1[3, :, :, 0]
B1 += shim_array[4, 0] * np.exp(1j * shim_array[4, 1]) * phantom.B1[4, :, :, 0]
B1 += shim_array[5, 0] * np.exp(1j * shim_array[5, 1]) * phantom.B1[5, :, :, 0]
B1 += shim_array[6, 0] * np.exp(1j * shim_array[6, 1]) * phantom.B1[6, :, :, 0]
B1 += shim_array[7, 0] * np.exp(1j * shim_array[7, 1]) * phantom.B1[7, :, :, 0]

plt.figure(figsize=(8, 4))
plt.subplot(121)
plt.title("abs(shim)")
plt.imshow(B1.abs().T, origin="lower", vmin=0)
plt.colorbar()
plt.axis("off")
plt.subplot(122)
plt.title("angle(shim)")
plt.imshow(B1.angle().T, origin="lower", vmin=-np.pi, vmax=np.pi, cmap="twilight")
cbar = plt.colorbar(ticks=[-np.pi, -np.pi/2, 0, np.pi/2, np.pi])
cbar.ax.set_yticklabels(["$-\\pi$", "$-\\pi/2$", "$0$",  "$\\pi/2$", "$\\pi$"])
plt.axis("off")
plt.show()